In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
genre = ["Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

In [6]:
data = pd.read_csv('u.item', sep="|", header=None , encoding='latin-1')

In [7]:
data.shape

(1682, 24)

In [8]:
data.columns = ["index", "title", "releaseDate", "", "imdb"] + genre +[""]

In [9]:
data['title']

0                                Toy Story (1995)
1                                GoldenEye (1995)
2                               Four Rooms (1995)
3                               Get Shorty (1995)
4                                  Copycat (1995)
                          ...                    
1677                            Mat' i syn (1997)
1678                             B. Monkey (1998)
1679                         Sliding Doors (1998)
1680                          You So Crazy (1994)
1681    Scream of Stone (Schrei aus Stein) (1991)
Name: title, Length: 1682, dtype: object

In [10]:
data.drop('', axis='columns', inplace=True)

In [11]:
data.iloc[:,4:]

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1678,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1679,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1680,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
arr = []
for i in range(data.shape[0]):
  arr.append([])
data['Genre'] = arr
data.iloc[:,-1]

0       []
1       []
2       []
3       []
4       []
        ..
1677    []
1678    []
1679    []
1680    []
1681    []
Name: Genre, Length: 1682, dtype: object

In [13]:
for i in range(data.shape[0]):
  for j in range(4, data.shape[1]-1):
      if(data.iloc[i,j] == 1):
        data.iloc[i,-1].append(genre[j-4])
data.iloc[:,-1]

0       [Children's, Comedy, Crime]
1       [Adventure, Animation, War]
2                             [War]
3       [Adventure, Crime, Fantasy]
4       [Documentary, Fantasy, War]
                   ...             
1677                      [Fantasy]
1678                  [Sci-Fi, War]
1679              [Fantasy, Sci-Fi]
1680                        [Crime]
1681                      [Fantasy]
Name: Genre, Length: 1682, dtype: object

In [16]:
for i in range(4, data.shape[1]-1):
  data.drop(genre[i-4], axis='columns', inplace=True)

In [17]:
data

,index,title,releaseDate,imdb,Genre
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Children's, Comedy, Crime]"
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"[Adventure, Animation, War]"
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,[War]
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"[Adventure, Crime, Fantasy]"
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),"[Documentary, Fantasy, War]"
...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,[Fantasy]
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"[Sci-Fi, War]"
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),"[Fantasy, Sci-Fi]"
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,[Crime]


In [18]:
data.describe()

,index
count,1682.000000
mean,841.500000
std,485.695893
min,1.000000
25%,421.250000
50%,841.500000
75%,1261.750000
max,1682.000000


In [23]:
def combine_features(row):
  str = ""
  str += row['title']+' '
  for val in row['Genre']:
    str += val+' '
  return str[:-1]

In [24]:
data['combined_features'] = data.apply(combine_features, axis = 1)

In [25]:
data

,index,title,releaseDate,imdb,Genre,combined_features
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Children's, Comedy, Crime]",Toy Story (1995) Children's Comedy Crime
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"[Adventure, Animation, War]",GoldenEye (1995) Adventure Animation War
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,[War],Four Rooms (1995) War
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"[Adventure, Crime, Fantasy]",Get Shorty (1995) Adventure Crime Fantasy
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),"[Documentary, Fantasy, War]",Copycat (1995) Documentary Fantasy War
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,[Fantasy],Mat' i syn (1997) Fantasy
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"[Sci-Fi, War]",B. Monkey (1998) Sci-Fi War
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),"[Fantasy, Sci-Fi]",Sliding Doors (1998) Fantasy Sci-Fi
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,[Crime],You So Crazy (1994) Crime
